In [1]:
!pip install nltk transformers torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
# chatbot.py
import random

intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["hi", "hello", "hey", "halo"],
            "responses": ["Halo!", "Hai, ada yang bisa saya bantu?", "Hai juga!"]
        },
        {
            "tag": "goodbye",
            "patterns": ["bye", "see you", "goodbye", "sampai jumpa"],
            "responses": ["Sampai jumpa!", "Bye bye!", "Semoga harimu menyenangkan!"]
        },
        {
            "tag": "thanks",
            "patterns": ["thanks", "thank you", "terima kasih"],
            "responses": ["Sama-sama!", "Senang bisa membantu!", "Kapan saja!"]
        }
    ]
}

def get_response(user_input):
    user_input = user_input.lower()
    for intent in intents['intents']:
        for pattern in intent['patterns']:
            if pattern in user_input:
                return random.choice(intent['responses'])
    return "Maaf, saya belum mengerti. Bisa dijelaskan lagi?"


In [3]:
import requests

def get_weather(city):
    api_key = "0e4b51aace445a20ca8c52b384327078"  # Ganti dengan API Key yang benar
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    # Membuat URL lengkap dengan parameter kota dan API Key
    complete_url = f"{base_url}q={city}&appid={api_key}&units=metric"

    # Melakukan permintaan GET ke API
    response = requests.get(complete_url)
    data = response.json()  # Mengonversi response ke format JSON

    if response.status_code == 200:
        main_data = data["main"]
        weather_data = data["weather"][0]
        temperature = main_data["temp"]
        feels_like = main_data["feels_like"]
        humidity = main_data["humidity"]
        description = weather_data["description"]

        # Mengembalikan informasi cuaca
        return (f"Suhu di {city} adalah {temperature}°C, "
                f"terasa seperti {feels_like}°C dengan cuaca {description}. "
                f"Kelembaban: {humidity}%.")
    else:
        return f"Terjadi kesalahan saat mengambil data cuaca. Status code: {response.status_code}"

# Contoh penggunaan
print(get_weather("Semarang"))


Suhu di Semarang adalah 26.96°C, terasa seperti 30.52°C dengan cuaca overcast clouds. Kelembaban: 89%.


In [6]:
!pip install pyngrok


In [7]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [12]:
from chatbot import get_response

FileNotFoundError: [Errno 2] No such file or directory: 'intent_model.pkl'

In [23]:
from pyngrok import ngrok
import random
import datetime
import requests
import threading
import time
import streamlit as st


# Pastikan token benar dan bisa akses
ngrok.set_auth_token("2w8HPYtfFnX2rtpXuYdFIbOdhEh_4HF3VXkjYSbufwsovxKPM")

# Tentukan port 8501 untuk streamlit
public_url = ngrok.connect(8501)
print(f"✅ Akses chatbot kamu di sini: {public_url}")

# Fungsi untuk menyimpan log percakapan
def save_conversation_log(user_input, bot_response):
    with open("conversation_log.txt", "a") as log_file:
        log_file.write(f"{datetime.datetime.now()} - User: {user_input} - Bot: {bot_response}\n")

# API cuaca (gunakan API key dari OpenWeatherMap)
API_KEY = "b46e6fad8459cedb9b6d6bc6b40bc4d4"  # Ganti dengan API Key yang valid
BASE_URL = "http://api.openweathermap.org/data/2.5/weather?"

def get_weather(city="Semarang"):
    complete_url = f"{BASE_URL}q={city}&appid={API_KEY}&units=metric"
    response = requests.get(complete_url)
    data = response.json()

    if data["cod"] != "404":
        main_data = data["main"]
        weather_data = data["weather"][0]
        temperature = main_data["temp"]
        description = weather_data["description"]
        return f"Suhu di {city}: {temperature}°C dengan cuaca {description}."
    else:
        return "Maaf, saya tidak bisa mendapatkan data cuaca saat ini."

# Intent untuk chatbot
intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["hi", "hello", "hey", "halo", "greetings"],
            "responses": ["Halo!", "Hai, ada yang bisa saya bantu?", "Hai juga!"]
        },
        {
            "tag": "goodbye",
            "patterns": ["bye", "see you", "goodbye", "sampai jumpa"],
            "responses": ["Sampai jumpa!", "Bye bye!", "Semoga harimu menyenangkan!"]
        },
        {
            "tag": "thanks",
            "patterns": ["thanks", "thank you", "terima kasih"],
            "responses": ["Sama-sama!", "Senang bisa membantu!", "Kapan saja!"]
        },
        {
            "tag": "time",
            "patterns": ["what time is it", "jam berapa", "time", "waktu sekarang"],
            "responses": [
                "Saat ini adalah: " + datetime.datetime.now().strftime("%H:%M:%S"),
                "Sekarang waktu: " + datetime.datetime.now().strftime("%H:%M:%S")
            ]
        },
        {
            "tag": "weather",
            "patterns": ["what's the weather like", "weather", "how's the weather", "cuaca", "suhu"],
            "responses": [
                get_weather("Semarang"),  # Ganti dengan kota lain jika perlu
                get_weather("Jakarta"),
                get_weather("Bandung")
            ]
        },
        {
            "tag": "joke",
            "patterns": ["tell me a joke", "joke", "give me a joke"],
            "responses": [
                "Kenapa komputer selalu dingin? Karena sering di-reboot!",
                "Kenapa orang suka main game? Karena mereka selalu punya level yang lebih tinggi!",
                "Apa bedanya programmer dengan pemandu wisata? Programmer menemukan bug, pemandu wisata menemukan tempat wisata!"
            ]
        },
        {
            "tag": "date",
            "patterns": ["tanggal berapa", "hari ini tanggal berapa", "what's the date", "tanggal"],
            "responses": [
                "Hari ini adalah " + datetime.datetime.now().strftime("%A, %d %B %Y"),
                "Sekarang tanggal: " + datetime.datetime.now().strftime("%d/%m/%Y")
            ]
      },
      {
          "tag": "fallback_funny",
          "patterns": ["kamu sedang apa", "lagi apa", "lagi ngapain", "kamu sibuk?"],
          "responses": [
              "Lagi nemenin kamu nih 😄",
              "Ngobrol sama kamu dong!",
              "Nunggu kamu nanya sesuatu 😁",
              "Bot juga butuh cinta, lho~"
          ]
      }
    ]
}

# Fungsi untuk mendapatkan respons chatbot
def get_response(user_input):
    user_input = user_input.lower()
    for intent in intents['intents']:
        for pattern in intent['patterns']:
            if pattern in user_input:
                bot_response = random.choice(intent['responses'])
                save_conversation_log(user_input, bot_response)  # Simpan percakapan
                return bot_response
    fallback_responses = [
    "Hmm... bisa diulangin dengan kata lain?",
    "Saya belum paham itu, tapi saya terus belajar!",
    "Mungkin maksud kamu bisa dijelaskan lebih detail?"
]
    bot_response = random.choice(fallback_responses)
    return bot_response

# Perbarui isi chatbot.py
with open("chatbot.py", "w") as f:
    f.write('''
import random
import datetime
import requests
import pickle

# Load model & vectorizer
model = pickle.load(open("intent_model.pkl", "rb"))
vectorizer = pickle.load(open("vectorizer.pkl", "rb"))

# Fungsi untuk menyimpan log percakapan
def save_conversation_log(user_input, bot_response):
    with open("conversation_log.txt", "a") as log_file:
        log_file.write(f"{datetime.datetime.now()} - User: {user_input} - Bot: {bot_response}\\n")

# API cuaca
API_KEY = "b46e6fad8459cedb9b6d6bc6b40bc4d4"
BASE_URL = "http://api.openweathermap.org/data/2.5/weather?"

def get_weather(city="Semarang"):
    complete_url = f"{BASE_URL}q={city}&appid={API_KEY}&units=metric"
    response = requests.get(complete_url)
    data = response.json()

    if data["cod"] != "404":
        main_data = data["main"]
        weather_data = data["weather"][0]
        temperature = main_data["temp"]
        description = weather_data["description"]
        return f"Suhu di {city}: {temperature}°C dengan cuaca {description}."
    else:
        return "Maaf, saya tidak bisa mendapatkan data cuaca saat ini."

intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["hi", "hello", "hey", "halo", "greetings"],
            "responses": ["Halo!", "Hai, ada yang bisa saya bantu?", "Hai juga!"]
        },
        {
            "tag": "goodbye",
            "patterns": ["bye", "see you", "goodbye", "sampai jumpa"],
            "responses": ["Sampai jumpa!", "Bye bye!", "Semoga harimu menyenangkan!"]
        },
        {
            "tag": "thanks",
            "patterns": ["thanks", "thank you", "terima kasih"],
            "responses": ["Sama-sama!", "Senang bisa membantu!", "Kapan saja!"]
        },
        {
            "tag": "time",
            "patterns": ["what time is it", "jam berapa", "time", "waktu sekarang"],
            "responses": [
                "Saat ini adalah: " + datetime.datetime.now().strftime("%H:%M:%S"),
                "Sekarang waktu: " + datetime.datetime.now().strftime("%H:%M:%S")
            ]
        },
        {
            "tag": "weather",
            "patterns": ["what's the weather like", "weather", "how's the weather", "cuaca", "suhu"],
            "responses": [
                get_weather("Semarang"),
                get_weather("Jakarta"),
                get_weather("Bandung")
            ]
        },
        {
            "tag": "joke",
            "patterns": ["tell me a joke", "joke", "give me a joke"],
            "responses": [
                "Kenapa komputer selalu dingin? Karena sering di-reboot!",
                "Kenapa orang suka main game? Karena mereka selalu punya level yang lebih tinggi!",
                "Apa bedanya programmer dengan pemandu wisata? Programmer menemukan bug, pemandu wisata menemukan tempat wisata!"
            ]
        },
        {
            "tag": "date",
            "patterns": ["tanggal berapa", "hari ini tanggal berapa", "what's the date", "tanggal"],
            "responses": [
                "Hari ini adalah " + datetime.datetime.now().strftime("%A, %d %B %Y"),
                "Sekarang tanggal: " + datetime.datetime.now().strftime("%d/%m/%Y")
            ]
        },
        {
            "tag": "fallback_funny",
            "patterns": ["kamu sedang apa", "lagi apa", "lagi ngapain", "kamu sibuk?"],
            "responses": [
                "Lagi nemenin kamu nih 😄",
                "Ngobrol sama kamu dong!",
                "Nunggu kamu nanya sesuatu 😁",
                "Bot juga butuh cinta, lho~"
            ]
        }
    ]
}

# Fungsi klasifikasi intent
def classify_intent(text):
    vec = vectorizer.transform([text])
    return model.predict(vec)[0]

# Fungsi response baru
def get_response(user_input):
    intent_tag = classify_intent(user_input)
    for intent in intents['intents']:
        if intent['tag'] == intent_tag:
            response = random.choice(intent['responses'])
            save_conversation_log(user_input, response)
            return response

    fallback = "Maaf, saya belum paham maksudnya."
    save_conversation_log(user_input, fallback)
    return fallback
''')


# Buat file app.py versi fix
with open("app.py", "w") as f:
    f.write('''
import streamlit as st
import time
from chatbot import get_response

st.set_page_config(page_title="Chatbot Sederhana", layout="centered")
st.title("🤖 Chatbot Cerdas Cuaca & Canda")

# Inisialisasi sesi
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# Input dari pengguna
user_input = st.text_input("Kamu:", "")

if user_input:
    # Tampilkan input pengguna di history
    st.session_state.chat_history.append(("Kamu", user_input))

    # Simulasi bot sedang mengetik
    with st.spinner("Bot sedang mengetik..."):
        time.sleep(1.2)
        response = get_response(user_input)

    st.session_state.chat_history.append(("Bot", response))

# Tampilkan semua percakapan
st.markdown("---")
st.subheader("💬 Riwayat Percakapan")

for sender, msg in st.session_state.chat_history:
    if sender == "Kamu":
        st.markdown(f"**🧑 Kamu:** {msg}")
    else:
        st.markdown(f"**🤖 Bot:** {msg}")
''')


# Jalankan streamlit di background
def run():
    !streamlit run app.py &> /dev/null

thread = threading.Thread(target=run)
thread.start()
time.sleep(5)


✅ Akses chatbot kamu di sini: NgrokTunnel: "https://6b0b-34-10-132-168.ngrok-free.app" -> "http://localhost:8501"


In [21]:
import json
import pandas as pd

# Data intents
from chatbot import intents

data = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        data.append((pattern, intent['tag']))

df = pd.DataFrame(data, columns=["text", "label"])
df.to_csv("intent_dataset.csv", index=False)
print("✅ Dataset intent disimpan sebagai intent_dataset.csv")


✅ Dataset intent disimpan sebagai intent_dataset.csv


In [20]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle

# Load data
df = pd.read_csv("intent_dataset.csv")

# Bersihkan data yang kosong
df = df.dropna(subset=['patterns', 'tag'])  # buang baris yang ada NaN di kolom patterns atau tag

# Vectorize
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['patterns'])
y = df['tag']

# Train model
model = LogisticRegression()
model.fit(X, y)

# Save model dan vectorizer
pickle.dump(model, open("intent_model.pkl", "wb"))
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))
